# Trying to detect whether someone is wearing a face mask or not

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import os
import random
import pickle
import urllib.request

import matplotlib.pyplot as plt

### Data Collection/Preparation

In [ ]:
df = pd.read_csv('../../Resources/data/mask_detection/mask_labels.udt.csv')
df.head()

In [ ]:
df = df[df['path'].str.contains('samples')]
df.shape

In [ ]:
df['output'].hist()
plt.show()

In [ ]:
df['output'] = df['output'].replace(to_replace='medical_mask', value='mask')
df['output'] = df['output'].replace(to_replace='not_medical_mask', value='mask')

In [ ]:
df['output'].hist()

In [ ]:
MASK_DIR = '../../Resources/data/mask_detection/mask/'
NOMASK_DIR = '../../Resources/data/mask_detection/no_mask/'

In [ ]:
df.head()

In [ ]:
# ## Run ONCE to download images
# for i in range(len(df)):
#     try:
#         img_url = df.iloc[i]['imageUrl']
#         name = df.iloc[i]['path'].replace('.', '_') + '.jpg'
#         output = df.iloc[i]['output']

#         if output == 'mask':
#             urllib.request.urlretrieve(img_url, os.path.join(MASK_DIR, name))
#         else:
#             urllib.request.urlretrieve(img_url, os.path.join(NOMASK_DIR, name))
#     except Exception as e:
#         print('Error for {}th request: {}'.format(i, e))

### Preprocessing

In [ ]:
import mtcnn
from PIL import Image

In [ ]:
def extract_face(filename, detector, required_size=(160, 160)):
    image = Image.open(filename).convert('RGB')
    pixels = np.asarray(image)
    
    faces = []
    
    results = detector.detect_faces(pixels)
    if results == []:
        return 'None'
    
    else:
        for result in results:
            x1, y1, width, height = result['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height

            face = pixels[y1:y2, x1:x2]
            image = Image.fromarray(face)
            image = image.resize(required_size)
            faces.append(np.asarray(image))
        return faces

In [ ]:
detector = mtcnn.MTCNN()

In [ ]:
def get_dataset(which='masked'):
    if which == 'masked':
        masked_face_arrays = [extract_face(os.path.join(MASK_DIR, i), detector) for i in tqdm(os.listdir(MASK_DIR))]
        masked_face_arrays = [i for i in masked_face_arrays if i != 'None']
        masked_face_arrays = [j for i in masked_face_arrays for j in i]
        masked_face_labels = [1] * len(masked_face_arrays)
        
        return masked_face_arrays, masked_face_labels
    
    else:
        unmasked_face_arrays = [extract_face(os.path.join(NOMASK_DIR, i), detector) for i in tqdm(os.listdir(NOMASK_DIR))]
        unmasked_face_arrays = [i for i in unmasked_face_arrays if i != 'None']
        unmasked_face_arrays = [j for i in unmasked_face_arrays for j in i]
        unmasked_face_labels = [0] * len(unmasked_face_arrays)
        
        return unmasked_face_arrays, unmasked_face_labels

In [ ]:
x = extract_face('../../Resources/data/mask_detection/mask/samples_328.jpg', detector)
type(x)

In [ ]:
# masked_data, masked_labels = get_dataset()
# len(masked_data)

In [ ]:
# unmasked_data, unmasked_labels = get_dataset(which='unmasked')
# len(unmasked_data)

In [ ]:
# images = masked_data + unmasked_data
# labels = masked_labels + unmasked_labels

In [ ]:
# data = list(zip(images, labels))

In [ ]:
# random.shuffle(data)

In [ ]:
# images, labels = zip(*data)

In [ ]:
# images = np.asarray(images)
# images.shape

In [ ]:
# labels = np.asarray(labels)
# labels.shape

In [ ]:
# file = open('../../Resources/data/mask_detection/images.pkl', 'wb')
# pickle.dump(images, file)
# file.close()

In [ ]:
# file = open('../../Resources/data/mask_detection/labels.pkl', 'wb')
# pickle.dump(labels, file)
# file.close()

In [ ]:
import tensorflow as tf
import pickle
import numpy as np

In [ ]:
images = pickle.load(open('../../Resources/data/mask_detection/images.pkl', 'rb'))
labels = pickle.load(open('../../Resources/data/mask_detection/labels.pkl', 'rb'))

In [ ]:
images.shape, labels.shape

In [ ]:
def get_embeddings(model, img):
    try:
#         x = tf.keras.preprocessing.image.load_img(img, target_size=(160, 160))
        x = tf.keras.preprocessing.image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = tf.keras.applications.vgg16.preprocess_input(x)

        return model.predict(x).reshape(-1)
    
    except Exception as e:
        print(e)
        return np.zeros((2048, ))

In [ ]:
base_model = tf.keras.applications.VGG16(include_top=False,
                                            weights='imagenet',
                                            input_shape=(160, 160, 3))
base_model.trainable = False
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalMaxPool2D()
])
model.summary()

In [ ]:
face_pixels = images.astype('float32')
mean, std = face_pixels.mean(), face_pixels.std()
mean, std

In [ ]:
face_pixels = (face_pixels - mean) / std
face_pixels.shape

In [ ]:
embeddings = [get_embeddings(model, i) for i in face_pixels]

In [ ]:
len(embeddings)

In [ ]:
file = open('../../Resources/data/mask_detection/embeddings.pkl', 'wb')
pickle.dump(embeddings, file)
file.close()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [ ]:
svm = SVC(kernel='linear')
svm.fit(embeddings, labels)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
preds = svm.predict(embeddings)

In [ ]:
accuracy_score(labels, preds)

In [ ]:
maps = {
    1: 'mask',
    0: 'no_mask'
}

In [ ]:
def check_mask(input_img):
    
    maps = {
        1: 'mask',
        0: 'no_mask'
    }
    
    img = (input_img - input_img.mean()) / input_img.std()
    img = get_embeddings(model, img)
    preds = maps[svm.predict(img.reshape(1, -1))[0]]
    
    plt.imshow(input_img)
    plt.title(preds)
    plt.show()

In [ ]:
img = images[500]
check_mask(img)

In [ ]:
img = images[700]
check_mask(img)

In [ ]:
import cv2

In [ ]:
def get_face(frame, detector, required_size=(160, 160)):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pixels = np.asarray(image)
    
    faces = []
    
    results = detector.detect_faces(pixels)
    if results == []:
        return 'None'
    
    else:
        for result in results:
            x1, y1, width, height = result['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height

            face = pixels[y1:y2, x1:x2]
            image = Image.fromarray(face)
#             image = image.resize(required_size)
            faces.append(np.asarray(image))
        return faces

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
x, y = 50, 50
offset = 35
while True:
    ret, frame = cap.read()
    faces = get_face(frame, detector)
#     print(faces)
    
    for face in faces:
        face_pixels = cv2.resize(face, (160, 160))
        mask = check_mask(face_pixels)
        
        print(mask)
        cv2.putText(frame, str(lbl), (x,y+offset*idx), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()